# Figure 7 - Analysis

In [ ]:
import sys
sys.path.append("..")

from main import *
from zebrafish import *
from visualization import *

from scipy.stats import zscore

# Computing geometric mapping cutoff point

#### Computing mode wavelengths using their variograms

In [ ]:
vertices = np.load('../Files/tectum_vertices_right.npy') * 40 # Rescaling the coordinates
vertices = np.stack([vertices[:, 2], vertices[:, 1], vertices[:, 0]], axis=1)
eigenmodes = np.load('../Files/tectum_eigenmodes_right.npy')[1:]

In [ ]:
compute = False

if compute:
    bins = np.linspace(0, 300, 60, endpoint=True)
    wavelengths, variograms = [], []
    for i in tqdm(range(eigenmodes.shape[0])):
        variogram = compute_variogram(vertices, zscore(eigenmodes[i]), bins=bins, subsample=2500)
        variograms.append(variogram)
        wavelengths.append(get_wavelength(variogram, bins))
    np.save('../Results/wavelengths_tectum.npy', wavelengths)
else:
    wavelengths = np.load('../Results/wavelengths_tectum.npy')

In [ ]:
half_wavelengths = np.array(wavelengths) / 2

#### Evaluating cutoff point in mode correlations

In [ ]:
from scipy.optimize import curve_fit

def compute_r_squared(y_data, y_model):
    ss_res = np.sum((y_data - y_model) ** 2)  # Residual sum of squares
    ss_tot = np.sum((y_data - np.mean(y_data)) ** 2)  # Total sum of squares
    r_squared = 1 - (ss_res / ss_tot)
    return r_squared

def piecewise_linear(x, x_break, slope, intercept):
    y1 = slope * (x - x_break) + intercept
    y2 = intercept
    return np.where(x < x_break, y1, y2)

#### Fitting a piecewise linear function to estimate cutoff point

In [ ]:
mode_similarity = np.load('../Results/figure7_similarity_matrix_50.npy')

In [ ]:
diagonal = np.diag(np.abs(mode_similarity))

In [ ]:
x = np.arange(len(diagonal))
y = diagonal

In [ ]:
initial_guess = [30, -1, 0]
params, _ = curve_fit(piecewise_linear, x, y, p0=initial_guess)

In [ ]:
print(params)

In [ ]:
plt.plot(x, y)
plt.plot(x, piecewise_linear(x, params[0], params[1], params[2]))

In [ ]:
rsquared_piecewise = compute_r_squared(diagonal, piecewise_linear(x, params[0], params[1], params[2]))

In [ ]:
np.save('../Results/figure7_diagonal_correlations.npy', diagonal)
np.save('../Results/figure7_piecewise_fit.npy', piecewise_linear(x, params[0], params[1], params[2]))

# Analyzing single-cell morphologies

In [ ]:
def values_to_colormap(values, cmap='viridis'):
    norm = plt.Normalize(vmin=np.min(values), vmax=np.max(values))
    colormap = plt.get_cmap(cmap)
    colors = [colormap(norm(value))[:3] for value in values]
    return colors

In [ ]:
atlas_path = '/home/anleg84/Documents/Atlas/Mapzebrain/'
atlas = Mapzebrain(atlas_path)

neurons = Neurons(atlas) # Class that loads all neuron morphologies

In [ ]:
somas = atlas.trim_centroids(neurons.somas) # Make sure the coordinates are within the atlas XYZ field of view
somas = np.round(somas).astype('int')

In [ ]:
mask_tectum = atlas.get_region_mask(22)
mask_neuropil = atlas.get_region_mask(17)

In [ ]:
neurons_in_tectum = np.where(mask_tectum[somas[:, 2], somas[:, 1], somas[:, 0]] > 0)[0]

Plotting tectal neuron morphologies

In [ ]:
#colors = generate_n_colors(10) * 1000
colors = values_to_colormap(somas[neurons_in_tectum, 1], cmap='rainbow') # Coloring neurons based on their antero-posterior location

fig, ax = plt.subplots(figsize=(10, 7))

ax.imshow(atlas.XYprojection, cmap='gray', alpha=0)
ax.axis('off')
for j, i in enumerate(neurons_in_tectum):
    neurons.plot_top(ax, i, color=colors[j], alpha=0.5, soma_size=5)

#ax.set_xlim([70, 500])
#ax.set_ylim([500, 250])
plt.tight_layout(pad=0)
plt.show()

In [ ]:
#colors = generate_n_colors(10) * 1000
colors = values_to_colormap(somas[neurons_in_tectum, 1], cmap='rainbow')

fig, ax = plt.subplots(figsize=(10, 7))

ax.imshow(atlas.XYprojection, cmap='gray', alpha=0)
ax.axis('off')
for j, i in enumerate(neurons_in_tectum):
    neurons.plot_top(ax, i, color=colors[j], alpha=0.5, soma_size=50)

ax.set_xlim([70, 500])
ax.set_ylim([500, 250])
plt.tight_layout(pad=0)
plt.show()

# Comparing morphologies with cutoff point

#### Loading linear fit data from tectal simulations and predicting connectivity radius

In [ ]:
x = np.load('../Results/supp_tectum_cutoff_x2.npy')
y = np.load('../Results/supp_tectum_cutoff_y2.npy')

In [ ]:
(a, b), CI_a, CI_b = linear_regression_with_confidence_interval(x, y, n_iter=100000)

In [ ]:
cutoff_wavelength = wavelengths[cutoff]

In [ ]:
print(cutoff_wavelength)

In [ ]:
predicted_radius = (cutoff_wavelength - b) / a
print(predicted_radius)

In [ ]:
radius_upper = (cutoff_wavelength - CI_b[0]) / CI_a[0]
radius_lower = (cutoff_wavelength - CI_b[1]) / CI_a[1]

In [ ]:
print(radius_lower)

In [ ]:
print(radius_upper)

#### Method #1: Measuring arborescence radius

In [ ]:
d = np.random.uniform(0, 1, (50, 3))

In [ ]:
radii = []
for i in tqdm(neurons_in_tectum):
    terminals = neurons.terminals[neurons.ids_terminals == i]
    c = np.round(terminals).astype('int')
    terminals_in_neuropil = np.where(mask_neuropil[c[:, 2], c[:, 1], c[:, 0]] > 0)[0]
    if np.any(terminals_in_neuropil):
        centroids = c[terminals_in_neuropil]
        center_of_mass = np.mean(centroids, axis=0)
        distances = np.sqrt(np.sum((centroids - center_of_mass) ** 2, axis=1))
        radius = np.percentile(distances, 95)
        radii.append(radius)
    else:
        radii.append(0)

radii = np.array(radii)[np.array(radii) >= 2]

In [ ]:
print(np.median(np.array(radii) * 2))

Plotting radius distribution along with predicted radius

In [ ]:
fig, ax = plt.subplots(figsize=(4, 2), dpi=150)
plt.hist(np.array(radii) * 2, bins=50, density=True, color='black')
ax.axvline(predicted_radius, color='red')
ax.spines[['top', 'right']].set_visible(False)
ax.set_xlabel('Connectivity radius (microns)')
ax.set_ylabel('Density')
ax.set_xlim([0, 300])
plt.show()

In [ ]:
np.save('../Results/figure7_predicted_radius.npy', predicted_radius)
np.save('../Results/figure7_predicted_radius_interval.npy', [radius_lower, radius_upper])
np.save('../Results/figure7_connectivity_radii_method1.npy', np.array(radii) * 2)

#### Method #2: Pseudo-synaptic partners

In [ ]:
N = len(neurons_in_tectum)

In [ ]:
connected = np.zeros((N, N))
for i, n1 in tqdm(enumerate(neurons_in_tectum)):
    for j, n2 in enumerate(neurons_in_tectum):
        if i != j:
            t1 = np.round(neurons.terminals[neurons.ids_terminals == n1]).astype('int')
            t2 = np.round(neurons.terminals[neurons.ids_terminals == n2]).astype('int')
            distances = compute_distances(t1, t2)
            if np.any(distances <= 5):
                connected[i, j] = 1

In [ ]:
somas = neurons.somas[neurons_in_tectum]

In [ ]:
distances = compute_distances(somas, somas)

In [ ]:
triangle = np.triu_indices(N, 1)

connected_distances = distances[triangle][connected[triangle] > 0]

In [ ]:
fig, ax = plt.subplots(figsize=(4, 2), dpi=150)
plt.hist(connected_distances, bins=50, density=True, color='black')
ax.axvline(predicted_radius, color='red')
ax.spines[['top', 'right']].set_visible(False)
ax.set_xlabel('Connectivity radius (microns)')
ax.set_ylabel('Density')
plt.show()

In [ ]:
C = connected * distances

In [ ]:
connected_radii = np.percentile(C, 95, axis=1)
connected_radii = connected_radii[connected_radii > 0]

In [ ]:
np.mean(connected_radii)

In [ ]:
fig, ax = plt.subplots(figsize=(4, 2), dpi=150)
plt.hist(connected_radii, bins=20, density=True, color='black')
ax.axvline(predicted_radius, color='red')
ax.spines[['top', 'right']].set_visible(False)
ax.set_xlabel('Connectivity radius (microns)')
ax.set_ylabel('Density')
ax.set_ylim([0, 0.02])
ax.set_xlim([0, 300])
plt.show()

In [ ]:
np.save('../Results/figure7_connectivity_radii_method2.npy', connected_radii)